In [46]:
!pip install keras-nlp --upgrade
#!pip install tensorflow==2.12.0

     -------------------------------------- 272.8/272.8 MB 5.9 MB/s eta 0:00:00
     --------------------------------------- 14.6/14.6 MB 23.4 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 19.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ------------------------------------- 420.6/420.6 kB 25.7 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 25.7 MB/s eta 0:00:00
     ------------------------------------- 440.7/440.7 kB 26.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 22.1 MB/s eta 0:00:00
     ---------------------------------------- 98.1/98.1 kB ? eta 0:00:00
  Created wheel for jax: filename=jax-0.

ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\jpmag\\.conda\\envs\\ap\\Lib\\site-packages\\~rapt\\_wrappers.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)
from tensorflow import keras
from keras.layers import *
import keras_nlp
from keras import backend

import re
import requests
import numpy as np
import random
import math

In [3]:
datafile = open("Shakespeare.txt", "r", encoding="utf-8")
text = datafile.read()
datafile.close()

In [4]:
vocab = sorted(list(set(text)))
embed_dim = math.ceil(math.log2(len(vocab)+1))

In [20]:
mask = 0
stoi = {ch:i+1 for i,ch in enumerate(vocab)}
itos = {i+1:ch for i,ch in enumerate(vocab)}
tokenize = lambda s: [stoi[c] for c in s]
detokenize = lambda l: "".join([itos[i] for i in l])
tokenized_text = tokenize(text)
text_size = len(tokenized_text)
vocab_size = len(vocab)
print(tokenized_text[:20])

[19, 48, 57, 58, 59, 2, 16, 48, 59, 48, 65, 44, 53, 11, 1, 15, 44, 45, 54, 57]


In [25]:
def get_train_sample(input_size):
    end = random.randint(0, text_size)
    start = max(end - input_size, 0)
    sample = tokenized_text[start : end]
    
    if (input_size - len(sample)) > 0:
        sample = [mask] * (input_size - len(sample)) + sample
    
    return sample


def get_train_epoch(epoch_size, input_size):
    X = []
    for _ in range(epoch_size):
        x = get_train_sample(input_size)
        X.append(x)
    X = tf.reshape(tf.constant(X), (len(X), input_size))
    return X

In [26]:
get_train_epoch(2, 5)

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[26, 64,  2, 59, 54],
       [52, 44,  2, 45, 54]])>

In [60]:
def build_model(input_size=128, num_blocks=5, embed_dim=64):
    inputs = Input(shape=(input_size,))
    embs = Embedding(vocab_size + 1, embed_dim)(inputs)
    x = embs + keras_nlp.layers.PositionEmbedding(input_size)(embs)
    
    for _ in range(num_blocks):
        x = x + MultiHeadAttention(
            int(math.sqrt(embed_dim)),
            int(math.sqrt(embed_dim))
        )(x, x, use_causal_mask=True)
        x = LayerNormalization()(x)
        x1 = Dense(embed_dim * 4, activation="leaky_relu")(x)
        x1 = Dense(embed_dim, activation="leaky_relu")(x1)
        x = LayerNormalization()(x + x1)

    x = Dense(embed_dim, activation="leaky_relu")(x)
    x = Dense(vocab_size + 1, activation="softmax")(x)
    
    model = keras.Model(inputs=inputs, outputs=x)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        metrics=["accuracy"]
    )
    return model

input_size = 128
model = build_model(input_size=input_size, num_blocks=5, embed_dim=64)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 128, 64)      4224        ['input_4[0][0]']                
                                                                                                  
 position_embedding_3 (Position  (None, 128, 64)     8192        ['embedding_3[0][0]']            
 Embedding)                                                                                       
                                                                                                  
 tf.__operators__.add_23 (TFOpL  (None, 128, 64)     0           ['embedding_3[0][0]',      

In [61]:
for i in range(100):
    X = get_train_epoch(4096*8, input_size + 1)
    model.fit(X[:,:-1], X[:,1:], shuffle=True, batch_size=256, epochs=1)

 70/128 [===============>..............] - ETA: 4:00 - loss: 2.4628 - accuracy: 0.2821

KeyboardInterrupt: 

In [62]:
text = """First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their"""

print(text, end="")
for _ in range(256):
    tokens = tokenize(text)
    tokens = tokens[max(0, len(tokens) - input_size):]
    tokens = [mask] * (input_size - len(tokens)) + tokens
    
    tokens = tf.reshape(tf.constant(tokens), (1, input_size))
    output = model(tokens)[0][-1]
    
    top_k = tf.math.top_k(output, 5)
    output = np.random.choice(top_k.indices, p=top_k.values.numpy()/np.sum(top_k.values.numpy()))
    if output == 0 or output > vocab_size:
        print("\nSTOPED\n")
        break
    output = detokenize([output])
    print(output, end="")
    text = text + "" + output

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise theire thim,
Tho he t andous that sengr s tene atoul tharereal myoulle me thin h has t s m thango aras tero st herile s
Thereas se tongreso ses theraneal tof h m thoras t thenoure as t t arerengrillo s thinge hangheal m ton hen s a

KeyboardInterrupt: 